In [1]:
from tqdm import tqdm
import pandas as pd
import json
import csv
import os

In [2]:
TEMP_CSV_PATH = "temp.csv"

In [3]:
# 읽어온 각각의 json을 필요한 정보만 뽑아서 row 하나로 만들기
def generate_csv_from_json_files(files):
    data = []
    for single_file in tqdm(files):
        with open(single_file, 'r', encoding='utf-8') as f:
            # Use 'try-except' to skip files that may be missing data
            try:
                json_file = json.load(f)
                file_id = json_file["데이터셋 정보"]["파일 번호"]
                file_name = json_file["데이터셋 정보"]["파일 이름"]
                top = json_file["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"]["상의"][0] # 상의만 가져올 것임
                data.append([file_id, file_name, top])
            except KeyError:
                print(f'Skipping {single_file}')
    # Add header
    data.insert(0, ['ID', 'File Name', 'Top'])
    print("\nREADING EACH JSON FILE ALL DONE! Length of data is", len(data))

    # csv로 변환
    print("Saving data as CSV file...")
    with open(TEMP_CSV_PATH, "w", encoding='utf-8', newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)
    print(f"CSV saved in {TEMP_CSV_PATH}")

In [4]:
def reduce_data(csv):
    # csv칼럼 내부 json 객체 flatten하기
    # https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json
    print("flattening columns...")
    TOP = pd.json_normalize(csv["Top"])
    flattened = csv[['ID', 'File Name']].join(TOP)

    # 우리 모델에 필요한 칼럼만 남기기
    print("dropping columns we don't need...")
    dropped = flattened[['ID', 'File Name', '카테고리','소매기장']]

    # one-hot 인코딩 하기
    print("one-hot encoding...")
    pd.set_option('display.max_columns', None)
    targets = dropped.columns[2:]
    final = pd.get_dummies(dropped, columns=targets, prefix_sep="_")

    # 기존에 사용한 파일 이제 무쓸모니까 삭제하기
    if(os.path.exists(TEMP_CSV_PATH) and os.path.isfile(TEMP_CSV_PATH)):
        os.remove(TEMP_CSV_PATH)
        print(f"{TEMP_CSV_PATH} file deleted")
    else:
        print(f"{TEMP_CSV_PATH} file not found")
    
    print("ALL DONE!")
    return final

In [5]:
def save_csv(csv, path):
    csv.to_csv(path,
                index = False,
                encoding='utf-8-sig')
    print(f"CSV Saved in {path}")